# Import

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
import random
from os import path

SEED = 20033002
random.seed(SEED)
np.random.seed(SEED)

# Data preprocessing

In [17]:
AUX_DATA_ROOT = "./data"
train = pd.read_csv(path.join(AUX_DATA_ROOT, 'train.csv'), index_col='id')

target = 'Цена'
train_idx = train.shape[0]
Y = train[target]

train.head()

,Тип_жилья,Широта,Долгота,Город,Индекс,Площадь,Этаж,Размер_участка,Расход_тепла,Ктгр_энергоэффективности,...,Нлч_парковки,Нлч_почтового_ящика,Нлч_балкона,Нлч_террасы,Нлч_подвала,Нлч_гаража,Нлч_кондиционера,Последний_этаж,Верхний_этаж,Цена
id,,,,,,,,,,,,,,,,,,,,,
35831305,квартира,48.875375,2.483820,rosny-sous-bois,4575,64.0,2.0,NaN,150.0,C,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,295000.0
35854039,квартира,45.522327,4.869322,vienne,1793,75.0,NaN,NaN,220.0,D,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,194000.0
36058437,квартира,43.672828,1.281469,mondonville,1425,42.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,179000.0
35989147,дом,44.158752,1.532081,caussade,4077,102.0,NaN,2000.0,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,205000.0
36029269,квартира,47.655212,-2.754196,vannes,2498,84.0,3.0,NaN,NaN,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,508000.0


## Filling NaN

In [18]:
def fill_na(df):    
    # custom filling
    means = train.groupby(by='Тип_жилья')['Размер_участка'].mean().isna()
    zero_types = means.index[means.values]
    df.loc[df['Тип_жилья'].isin(zero_types), 'Размер_участка'] = 0
    
    means = train.groupby(by='Тип_жилья')['Расход_тепла'].mean().isna()
    zero_types = means.index[means.values]
    df.loc[df['Тип_жилья'].isin(zero_types), ['Расход_тепла', 'Кво_вредных_выбросов']] = 0

    mode = train.groupby(by='Тип_жилья')['Ктгр_энергоэффективности'].agg(lambda x: x.mode())
    empty_indices = [i for i, x in enumerate(mode.values) if isinstance(x, np.ndarray) and len(x) == 0]
    unk_types = mode.index[empty_indices]
    df.loc[df['Тип_жилья'].isin(unk_types), ['Ктгр_энергоэффективности', 'Ктгр_вредных_выбросов']] = 'unkown'
    
    mode = train.groupby(by='Тип_жилья')['Направление'].agg(lambda x: x.mode())
    empty_indices = [i for i, x in enumerate(mode.values) if isinstance(x, np.ndarray) and len(x) == 0]
    unk_types = mode.index[empty_indices]
    df.loc[df['Тип_жилья'].isin(unk_types), 'Направление'] = 'unkown'
    
    df.fillna({'Кво_фото': 0}, inplace=True)
    df.loc[(df['Тип_жилья'] == 'комната') & df['Кво_спален'].isna(), 'Кво_спален'] = 1
    
    nan_num_cols = ['Площадь', 'Кво_комнат', 'Кво_спален', 'Размер_участка','Расход_тепла', 'Кво_вредных_выбросов']
    med_groups = train.groupby('Тип_жилья')[nan_num_cols].median()

    for col in nan_num_cols:
        df[col] = df.apply(
            lambda row: med_groups[col][row['Тип_жилья']] if pd.isna(row[col]) else row[col],
            axis=1)
    
    # simple filling
    si_n = SimpleImputer(strategy='median')
    si_c = SimpleImputer(strategy='most_frequent')
    
    num_cols = df.select_dtypes(include='number').columns
    si_n.fit(train[num_cols])
    df[num_cols] = si_n.transform(df[num_cols])
    
    cat_cols = df.select_dtypes(include='O').columns
    si_c.fit(train[cat_cols])
    df[cat_cols] = si_c.transform(df[cat_cols])
    
    return df

train = fill_na(train)

df = train.drop(columns=target).copy()
df.isna().sum().sum()

0

## EDA

### Logarithmic and count features

In [19]:
def log_nan(row):
    row[row < 1] = 1
    return np.log(row)

df_ext = df.copy()
cols_to_log = ['Площадь', 'Размер_участка', 'Расход_тепла', 'Кво_вредных_выбросов',]
log_cols = ['Лог_' + col for col in cols_to_log]
df_ext[log_cols] = df[cols_to_log].apply(log_nan)

In [20]:
bool_cols = ['Нлч_парковки', 'Нлч_почтового_ящика', 'Нлч_балкона', 'Нлч_террасы', 'Нлч_подвала', 
            'Нлч_гаража', 'Нлч_кондиционера']
df_ext['Плюшки'] = df_ext[bool_cols].sum(axis=1)

df_ext.head()

,Тип_жилья,Широта,Долгота,Город,Индекс,Площадь,Этаж,Размер_участка,Расход_тепла,Ктгр_энергоэффективности,...,Нлч_подвала,Нлч_гаража,Нлч_кондиционера,Последний_этаж,Верхний_этаж,Лог_Площадь,Лог_Размер_участка,Лог_Расход_тепла,Лог_Кво_вредных_выбросов,Плюшки
id,,,,,,,,,,,,,,,,,,,,,
35831305,квартира,48.875375,2.483820,rosny-sous-bois,4575.0,64.0,2.0,94.5,150.0,C,...,1.0,0.0,0.0,0.0,0.0,4.158883,4.548600,5.010635,2.484907,2.0
35854039,квартира,45.522327,4.869322,vienne,1793.0,75.0,2.0,94.5,220.0,D,...,0.0,0.0,0.0,0.0,0.0,4.317488,4.548600,5.393628,3.931826,0.0
36058437,квартира,43.672828,1.281469,mondonville,1425.0,42.0,2.0,94.5,173.0,D,...,0.0,0.0,0.0,0.0,0.0,3.737670,4.548600,5.153292,2.639057,1.0
35989147,дом,44.158752,1.532081,caussade,4077.0,102.0,2.0,2000.0,188.0,D,...,0.0,0.0,0.0,0.0,0.0,4.624973,7.600902,5.236442,2.890372,1.0
36029269,квартира,47.655212,-2.754196,vannes,2498.0,84.0,3.0,94.5,173.0,D,...,1.0,0.0,0.0,0.0,0.0,4.430817,4.548600,5.153292,2.639057,3.0


### Cluster features

In [21]:
kmeans = KMeans(n_clusters=15, n_init=10, random_state=SEED)
kmeans.fit(train[['Широта', 'Долгота']])
df_ext['Кластер'] = kmeans.predict(df_ext[['Широта', 'Долгота']])
df_ext['Кластер'] = df_ext['Кластер'].astype("object")

df_cd = kmeans.transform(df_ext[['Широта', 'Долгота']])

df_cd = pd.DataFrame(df_cd, columns=[f"Центр_{i}" for i in range(df_cd.shape[1])], index=df_ext.index)
df_ext = df_ext.join(df_cd)
df_ext.head()

,Тип_жилья,Широта,Долгота,Город,Индекс,Площадь,Этаж,Размер_участка,Расход_тепла,Ктгр_энергоэффективности,...,Центр_5,Центр_6,Центр_7,Центр_8,Центр_9,Центр_10,Центр_11,Центр_12,Центр_13,Центр_14
id,,,,,,,,,,,,,,,,,,,,,
35831305,квартира,48.875375,2.483820,rosny-sous-bois,4575.0,64.0,2.0,94.5,150.0,C,...,5.968955,5.326387,4.204772,5.591455,7.224722,5.289429,3.998422,1.311812,2.362927,2.633328
35854039,квартира,45.522327,4.869322,vienne,1793.0,75.0,2.0,94.5,220.0,D,...,1.940456,8.075758,0.728199,5.616398,3.144573,3.782871,4.200185,4.804905,2.194875,4.979323
36058437,квартира,43.672828,1.281469,mondonville,1425.0,42.0,2.0,94.5,173.0,D,...,4.078129,6.008742,4.761215,1.945976,5.886928,0.292558,1.745806,6.603585,3.322162,3.901142
35989147,дом,44.158752,1.532081,caussade,4077.0,102.0,2.0,2000.0,188.0,D,...,3.861773,5.843233,4.321843,2.122949,5.683645,0.495724,1.435327,6.067547,2.776054,3.515600
36029269,квартира,47.655212,-2.754196,vannes,2498.0,84.0,3.0,94.5,173.0,D,...,9.050716,0.442123,8.412050,4.072727,10.801720,5.886933,4.154200,6.352098,5.792370,3.036566


### PCA

In [22]:
def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y, random_state=SEED)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [23]:
pca_cols = ['Площадь', 'Размер_участка', 'Расход_тепла', 'Кво_вредных_выбросов',
            'Лог_Площадь', 'Лог_Размер_участка', 'Лог_Расход_тепла', 'Лог_Кво_вредных_выбросов', 
            'Кво_комнат', 'Кво_спален', 'Кво_ванных',]

train_pca = df_ext.iloc[:train_idx][pca_cols].copy()
train_pca_scaled = (train_pca - train_pca.mean(axis=0)) / train_pca.std(axis=0)
pca = PCA(random_state=SEED)

X_pca = pca.fit_transform(train_pca_scaled)
component_names = [f"PC{i}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

pca_mi = make_mi_scores(X_pca, train[target])
pca_mi

PC0     0.191245
PC1     0.169319
PC4     0.156414
PC8     0.144177
PC9     0.138456
PC5     0.135458
PC3     0.135070
PC2     0.126248
PC6     0.122356
PC10    0.103659
PC7     0.099129
Name: MI Scores, dtype: float64

In [24]:
top_pca = list(pca_mi.index[:5])
df_pca = df_ext[pca_cols].copy()
df_pca_scaled = (df_pca - df_pca.iloc[:train_idx].mean(axis=0)) / df_pca.iloc[:train_idx].std(axis=0)
df_pca = pca.transform(df_pca_scaled)

df_ext[top_pca] = df_pca[:, [int(pca_t[2:]) for pca_t in top_pca]]

In [25]:
pca_cols_c = ['Центр_0', 'Центр_1','Центр_2', 'Центр_3', 
            'Центр_4', 'Центр_5', 'Центр_6', 'Центр_7',
            'Центр_8', 'Центр_9', 'Центр_10', 'Центр_11', 
            'Центр_12', 'Центр_13', 'Центр_14',]


train_pca = df_ext.iloc[:train_idx][pca_cols_c].copy()
pca = PCA(random_state=SEED)

X_pca = pca.fit_transform(train_pca)

component_names = [f"PC_ц{i}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

pca_mi = make_mi_scores(X_pca, train[target])
pca_mi

PC_ц0     0.234372
PC_ц5     0.213921
PC_ц1     0.199394
PC_ц4     0.187374
PC_ц9     0.169992
PC_ц3     0.169297
PC_ц2     0.169057
PC_ц11    0.167679
PC_ц10    0.167003
PC_ц14    0.166981
PC_ц6     0.154049
PC_ц13    0.153805
PC_ц7     0.149320
PC_ц12    0.148683
PC_ц8     0.135141
Name: MI Scores, dtype: float64

In [26]:
top_pca_c = list(pca_mi.index[:10])
df_pca = pca.transform(df_ext[pca_cols_c])

df_ext[top_pca_c] = df_pca[:, [int(pca_t[4:]) for pca_t in top_pca_c]]

## Outliers processing

In [27]:
outls_clf = IsolationForest(contamination=0.01, random_state=SEED, max_features=0.5, n_estimators=500)
cat_cols = df_ext.select_dtypes(include='O').columns
anomal_ind = outls_clf.fit_predict(df_ext.drop(columns=cat_cols)[:train_idx])
anomal_ind = anomal_ind == 1
train_clean = df_ext.iloc[:train_idx].loc[anomal_ind]
df_ext = pd.concat((train_clean, df_ext[train_idx:]))

train_idx_old = train_idx
train_idx = train_clean.shape[0]
print(f'Total outliers in the data: {train_idx_old - train_idx}')

Total outliers in the data: 262


## Deleting bad features

In [28]:
# these columns had low feature importance value in CatBoost 
drop_cols = [
    'Последний_этаж', 'Верхний_этаж', 'Этаж',
    'Центр_2', 'Центр_3', 'Центр_4', 'Центр_6', 'Центр_7'
    ]
df_ext.drop(columns=drop_cols, inplace=True)

# Models and training

In [29]:
class MyMAPE: # MAPE for catboost log preds
    @staticmethod
    def mape(y_t, y_p):
        return mape(y_t, y_p)
    
    def get_final_error(self, error, weight):
        return error
    
    def is_max_optimal(self):
        # the larger metric value the better
        return False

    def evaluate(self, y_p, y_t, weight):
        score = self.mape(np.exp(y_t), np.exp(y_p[0]))
        return score, 0

In [35]:
best_params = {
    'random_seed': SEED,
    'iterations': 10000,
    'eval_metric': MyMAPE(),
    'l2_leaf_reg': 2.81255,
    'learning_rate': 0.0721322,
    'loss_function': 'RMSE',
    'leaf_estimation_method': 'Newton',
    'bootstrap_type': 'MVS',
    'boosting_type': 'Plain',
    'grow_policy': 'Depthwise',
    'rsm': 0.4535,
    'depth': 10,
    'verbose': 5000,
    'task_type': 'CPU',
    }

## Беггинг (KFold)

In [18]:
kf = KFold(n_splits=6, shuffle=True, random_state=SEED)

y = np.log(Y)[anomal_ind]
X = df_ext[:train_idx]

cat_cols = list(df_ext.select_dtypes(include='O').columns)

models = []
weights = []

for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    train_pool = Pool(data=x_train, label=y_train, cat_features=cat_cols)
    val_pool = Pool(data=x_test, label=y_test, cat_features=cat_cols)

    model = CatBoostRegressor(**best_params)
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=300, use_best_model=True)

    # model.save_model(f'weights/cat_boost_{i}')
    models.append(model)
    weights.append(model.best_score_['validation']['MyMAPE'])
    
    l = 20
    print('^' * 3 + '-' * l + str(i + 1) + '-' * l + '^' * 3)

models = np.array(models)

mean_val_score = np.mean(weights)
print(f'Mean validation score = {mean_val_score:.6f}')
print('Validation MAPE score per model:', weights)

weights = [1/w for w in weights]
weights = np.array(weights) / np.sum(weights)
print('Weights of the models in the ensemble:', weights)

np.save('weights/bagging_models.npy', models, allow_pickle=True)
np.save('weights/vote_weights.npy', weights)
print('All weigths were saved in \'./weigts/\' directory')

/media/mikhail_solovev/Memory/py_proj/venvs/rucode_2024/lib/python3.12/site-packages/catboost/core.py:2321: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)


0:	learn: 0.7695524	test: 0.7774211	best: 0.7774211 (0)	total: 102ms	remaining: 17m 1s
5000:	learn: 0.0086617	test: 0.2641022	best: 0.2640981 (4983)	total: 2m 55s	remaining: 2m 55s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2639764578
bestIteration = 6854

Shrink model to first 6855 iterations.
^^^--------------------1--------------------^^^
0:	learn: 0.7630669	test: 0.7846887	best: 0.7846887 (0)	total: 54.8ms	remaining: 9m 7s


/media/mikhail_solovev/Memory/py_proj/venvs/rucode_2024/lib/python3.12/site-packages/catboost/core.py:2321: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)


Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2618208934
bestIteration = 2710

Shrink model to first 2711 iterations.
^^^--------------------2--------------------^^^
0:	learn: 0.7661961	test: 0.7775495	best: 0.7775495 (0)	total: 47.3ms	remaining: 7m 53s


/media/mikhail_solovev/Memory/py_proj/venvs/rucode_2024/lib/python3.12/site-packages/catboost/core.py:2321: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)


...

Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2617649384
bestIteration = 3535

Shrink model to first 3536 iterations.
^^^--------------------4--------------------^^^
0:	learn: 0.7725055	test: 0.7401618	best: 0.7401618 (0)	total: 47ms	remaining: 7m 50s


/media/mikhail_solovev/Memory/py_proj/venvs/rucode_2024/lib/python3.12/site-packages/catboost/core.py:2321: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)


5000:	learn: 0.0089096	test: 0.2623219	best: 0.2623179 (4976)	total: 2m 44s	remaining: 2m 44s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2622923772
bestIteration = 5254

Shrink model to first 5255 iterations.
^^^--------------------5--------------------^^^
0:	learn: 0.7692417	test: 0.7622960	best: 0.7622960 (0)	total: 66.1ms	remaining: 11m


/media/mikhail_solovev/Memory/py_proj/venvs/rucode_2024/lib/python3.12/site-packages/catboost/core.py:2321: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)


Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.2585170414
bestIteration = 2382

Shrink model to first 2383 iterations.
^^^--------------------6--------------------^^^
Mean validation score = 0.262208
Validation MAPE score per model: [0.2639764578091603, 0.26182089343081166, 0.2648740595889578, 0.26176493836531406, 0.262292377156238, 0.25851704137891396]
Weights of the models in the ensemble: [0.16554012 0.16690301 0.16497914 0.16693869 0.166603   0.16903604]
All weigths were saved in './weigts/' directory
